In [1]:
from neo4j import GraphDatabase

# Connecting to Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))

# Cypher query to extract interactions
query = """
MATCH (d1:Drug)-[r:CONTRADICTS]->(d2:Drug)
RETURN d1.name AS drug_a, d2.name AS drug_b, r.severity AS severity
"""

def fetch_interactions(driver):
    with driver.session() as session:
        result = session.run(query)
        return [(record['drug_a'], record['drug_b'], record['severity']) for record in result]

interactions = fetch_interactions(driver)
driver.close()

### Collaborative Filtering with Matrix Factorization (SVD)

In [2]:
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds

# Encoding severities
severity_mapping = {'Major': 3, 'Moderate': 2, 'Minor': 1, 'Unknown': 0}
encoded_interactions = [(d1, d2, severity_mapping.get(severity, 0)) for d1, d2, severity in interactions]

# Extracting unique drugs
drugs = list(set([d1 for d1, _, _ in encoded_interactions] + [d2 for _, d2, _ in encoded_interactions]))
drug_index = {drug: idx for idx, drug in enumerate(drugs)}

# Creating the interaction matrix
interaction_matrix = np.zeros((len(drugs), len(drugs)))
for d1, d2, severity in encoded_interactions:
    idx1, idx2 = drug_index[d1], drug_index[d2]
    interaction_matrix[idx1, idx2] = severity
    interaction_matrix[idx2, idx1] = severity  # assuming symmetry
interaction_df = pd.DataFrame(interaction_matrix, index=drugs, columns=drugs)

# Applying SVD
u, s, vt = svds(interaction_matrix, k=50)  # k is the number of latent factors
predicted_interactions = np.dot(np.dot(u, np.diag(s)), vt)
predicted_interactions_df = pd.DataFrame(predicted_interactions, index=drugs, columns=drugs)
predicted_interactions_df.head()

,Azelastine (nasal),Iron Dextran,Methimazole,Milk thistle,Vinorelbine,Risperidone,Vandetanib,Enasidenib,Ospemifene,Desirudin,...,Sodium oxybate,Flucytosine,Artesunate,Propofol,Cisplatin,Dinoprostone (topical),Mirabegron,Dapagliflozin,Frovatriptan,Pyridostigmine
Azelastine (nasal),0.989117,0.000627,-0.053089,0.000012,-0.017516,1.659138,-0.117919,-0.056214,0.010548,-0.191941,...,1.652190,0.025029,-0.083231,0.367083,-0.001636,0.001875,0.144285,-0.052977,0.382936,-0.093122
Iron Dextran,0.000627,0.010458,-0.006306,0.005777,0.018052,-0.007989,-0.009074,0.008429,-0.001806,-0.030509,...,-0.003602,0.010889,0.014408,0.004123,0.033441,0.001232,-0.005049,-0.014297,0.005377,-0.012564
Methimazole,-0.053089,-0.006306,0.024476,-0.005912,-0.141105,0.139268,-0.110688,-0.059220,0.144115,0.101146,...,0.010638,-0.116970,0.025981,-0.045933,0.045160,-0.004499,0.007669,-0.111808,-0.154313,0.148085
Milk thistle,0.000012,0.005777,-0.005912,0.001536,-0.001682,-0.003271,0.001112,-0.007772,-0.000336,-0.020894,...,-0.001426,0.010721,-0.001040,0.000320,-0.006022,0.000276,-0.002468,-0.001684,-0.001193,-0.015634
Vinorelbine,-0.017516,0.018052,-0.141105,-0.001682,2.718850,0.015426,0.281823,0.687992,0.151699,0.093472,...,0.000573,0.037754,0.036374,-0.096058,2.861579,-0.015952,0.175356,0.033071,0.042065,0.039281


 ### Applying a Random Forest Classifier to Predict Interaction Severity

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Creating training data
train_data = []
for d1, d2, severity in encoded_interactions:
    idx1, idx2 = drug_index[d1], drug_index[d2]
    features = np.concatenate([u[idx1], vt.T[idx2]])
    train_data.append((features, severity))

X, y = zip(*train_data)
X = np.array(X)
y = np.array(y)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluating the model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      5961
           1       0.98      0.73      0.83      1351
           2       0.94      0.98      0.96     19317
           3       0.96      0.88      0.92      5418

    accuracy                           0.95     32047
   macro avg       0.96      0.88      0.92     32047
weighted avg       0.95      0.95      0.95     32047



### Finding pairs of drugs for which the interaction severity is not reported

In [4]:
import os
from tools import Files

files = Files()
root = os.path.dirname(os.path.abspath(__name__))
drugs_list = files.read_json(os.path.join(root, "data", "processed", "drugs_list.json"))
interaction_pairs = set([(d1, d2) for d1, d2, _ in encoded_interactions])

sample_size = 1000
drug_pairs = []
while True:
    drug_a, drug_b = np.random.choice(drugs_list, 2, replace=False)
    if (drug_a, drug_b) in interaction_pairs or (drug_b, drug_a) in interaction_pairs:
        continue
    if (drug_a, drug_b) not in drug_pairs and (drug_b, drug_a) not in drug_pairs:
        drug_pairs.append((drug_a, drug_b))
    if len(drug_pairs) == sample_size:
        break

drug_pairs[:5]

[('Hydroxocobalamin', 'Turmeric'),
 ('Lonafarnib', 'Abarelix'),
 ('Grazoprevir', 'Naproxen'),
 ('Mesalazine', 'Oritavancin'),
 ('Oxcarbazepine', 'Methoxamine')]

In [5]:
def predict_interaction_rf(drug_a, drug_b):
    idx_a, idx_b = drug_index[drug_a], drug_index[drug_b]
    features = np.concatenate([u[idx_a], vt.T[idx_b]])
    predicted_severity = clf.predict([features])[0]
    severity_labels = {3: 'major', 2: 'moderate', 1: 'minor', 0: 'unknown'}
    return severity_labels[predicted_severity]

new_predictions = []
for drug_a, drug_b in drug_pairs:
    new_interaction = predict_interaction_rf(drug_a, drug_b)
    new_predictions.append((drug_a, drug_b, new_interaction))
new_predictions = pd.DataFrame(new_predictions, columns=['A', 'B', 'Severity'])
new_predictions.tail()

,A,B,Severity
995,Deferiprone,Halcinonide (topical),moderate
996,Cabergoline,Verapamil,unknown
997,Interferon alfacon-1,Anakinra,moderate
998,Aclidinium,Urofollitropin,unknown
999,Aminolevulinic acid (topical),Trichophyton mentagrophytes,moderate


### Applying Graph Neural Networks (GNNs)

#### Creating a Graph Data Object

In [6]:
import torch
from torch_geometric.data import Data
import networkx as nx

G = nx.Graph()
severity_mapping = {'Major': 3, 'Moderate': 2, 'Minor': 1, 'Unknown': 0}
for drug_a, drug_b, severity in interactions:
    G.add_edge(drug_a, drug_b, weight=severity_mapping.get(severity, 0))
drug_to_index = {drug: idx for idx, drug in enumerate(G.nodes)}

edges = [(drug_to_index[u], drug_to_index[v]) for u, v in G.edges]
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
edge_attr = torch.tensor([G[u][v]['weight'] for u, v in G.edges], dtype=torch.float).view(-1, 1)

# One-hot encoding of drug classes
# In future work, molecular descriptors or fingerprints can also be used as node features. 
num_drugs = len(G.nodes)
x = torch.eye(num_drugs)

data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
print(data)

Data(x=[1939, 1939], edge_index=[2, 160235], edge_attr=[160235, 1])


### Defining the Graph Neural Network (GNN) Model

#### 1. GNN Architecture

In [7]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

class EdgePredictor(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(EdgePredictor, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fc1 = torch.nn.Linear(hidden_channels * 2, hidden_channels)
        self.fc2 = torch.nn.Linear(hidden_channels, 1)  # Output layer for binary classification

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        
        # GCN layers to get node embeddings
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)  # Node embeddings
        
        # Extract embeddings for the pairs of nodes connected by edges
        edge_embeddings = self.get_edge_embeddings(x, edge_index)
        
        # Pass edge embeddings through the classifier
        edge_scores = self.fc1(edge_embeddings)
        edge_scores = F.relu(edge_scores)
        edge_scores = self.fc2(edge_scores)
        edge_scores = torch.sigmoid(edge_scores).squeeze()  # Final predictions
        
        return edge_scores  # Shape: [num_edges]
    
    def get_edge_embeddings(self, node_embeddings, edge_index):
        # edge_index has shape [2, num_edges]
        src_nodes = edge_index[0]  # Source nodes
        dst_nodes = edge_index[1]  # Destination nodes
        
        src_embeddings = node_embeddings[src_nodes]  # Shape: [num_edges, hidden_channels]
        dst_embeddings = node_embeddings[dst_nodes]  # Shape: [num_edges, hidden_channels]
        
        # Combine node embeddings to form edge embeddings (e.g., concatenation)
        edge_embeddings = torch.cat([src_embeddings, dst_embeddings], dim=1)  # Shape: [num_edges, hidden_channels*2]
        
        return edge_embeddings

#### 2. Training Loop for the GNN Model

In [8]:
import torch.optim as optim

# Set the device
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Move the Data object to the correct device
data = data.to(device)

# Extract labels after moving the data to the correct device
labels = data.edge_attr.squeeze()  # Assuming edge_attr is already on the correct device

# Initialize the model and move it to the correct device
model = EdgePredictor(num_node_features=data.x.shape[1], hidden_channels=64).to(device)

# Set up the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

def train():
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    out = model(data)  # Shape: [num_edges]
    
    # Calculate loss
    loss = criterion(out, labels.float())
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    return loss.item()

# Training the model
epochs = 200
for epoch in range(1, epochs + 1):
    loss = train()
    print(f'Epoch {epoch}, Loss: {loss}')

RuntimeError: all elements of target should be between 0 and 1

In [13]:
print(f"Unique Label Values: {labels.unique()}")

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### 3. Evaluating the GNN Model

In [45]:
def evaluate():
    model.eval()
    with torch.no_grad():
        out = model(data)
        predictions = (out >= 0.5).long()
        correct = (predictions == labels).sum().item()
        accuracy = correct / labels.size(0)
        return accuracy

accuracy = evaluate()
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


### 4. Predicting New Drug Interactions

In [46]:
def predict_interaction_gnn(drug_a, drug_b):
    model.eval()
    with torch.no_grad():
        # Getting indices of the drugs
        idx_a = drug_to_index[drug_a]
        idx_b = drug_to_index[drug_b]
        
        # Creating edge_index tensor for this pair
        edge_index = torch.tensor([[idx_a], [idx_b]], dtype=torch.long).to(device)

        # Create a temporary Data object
        # Since the model processes the entire graph, we'll use the existing node embeddings -> data.x
        temp_data = Data(x=data.x, edge_index=edge_index)
        temp_data = temp_data.to(device)
        
        # Get Prediction - Probability between 0 and 1
        prediction = model(temp_data).item()
        
        return prediction  

prediction_classes = {
    (0.7, 1): 'major',
    (0.4, 0.7): 'moderate',
    (0.1, 0.4): 'minor',
    (0, 0.1): 'unknown'
}
new_predictions = []
for drug_a, drug_b in drug_pairs:
    severity_significance = predict_interaction_gnn(drug_a, drug_b)
    for (lower, upper), severity in prediction_classes.items():
        if lower <= severity_significance < upper:
            new_predictions.append((drug_a, drug_b, severity, severity_significance))
            break
new_predictions = pd.DataFrame(new_predictions, columns=['A', 'B', 'Severity', 'Significance'])
new_predictions.head()

,A,B,Severity,Significance
0,Trichlormethiazide,Calcipotriol,unknown,3.081682e-33
1,Enalapril,Estazolam,unknown,6.035020e-33
2,Cetylpyridinium (topical),Valganciclovir,unknown,1.985426e-29
3,Sirolimus,Tipiracil,unknown,6.474631e-33
4,Racepinephrine,Magnesium carbonate,unknown,3.900321e-34


In [47]:
import numpy as np
print(np.bincount(labels.cpu().numpy().astype(int)))

[160235]


In [49]:
labels.unique()

tensor([0.], device='cuda:0')